# AILib Tutorial 8: Agents - Autonomous Problem Solving

Agents are the crown jewel of AILib - autonomous AI systems that can use tools to solve complex problems. In this tutorial, you'll learn:

- Creating basic agents
- The ReAct (Reasoning and Acting) pattern
- Providing agents with tools
- Controlling agent behavior
- Debugging and monitoring agents
- Building sophisticated agent systems

## Setup

Let's import what we need:

In [ ]:
from ailib import OpenAIClient, create_agent
from ailib.agents import Agent, Tool, ToolRegistry, tool
from ailib.prompts import PromptTemplate
from datetime import datetime
from dotenv import load_dotenv
import json
import math

# Load environment variables
load_dotenv()

# Create a client
client = OpenAIClient(model="gpt-4")  # Agents work best with GPT-4
print("Ready to create agents!")

## Basic Agent Creation

AILib provides two ways to create agents:

1. **Factory Function** (Recommended) - Simple and validated
2. **Direct Instantiation** - More control when needed

In [ ]:
# Method 1: Using the factory function (Recommended)
simple_agent = create_agent("assistant", model="gpt-4")

# Run the agent
response = simple_agent.run("What is the capital of France?")
print("Simple agent response:")
print(response)

# Create an agent with custom settings
custom_agent = create_agent(
    "assistant",
    model="gpt-4",
    max_steps=5,     # Maximum reasoning steps
    verbose=True,    # Show reasoning process
    temperature=0.7  # Control creativity
)

print("\n" + "="*50 + "\n")
print("Custom agent with verbose output:")
response = custom_agent.run("Explain photosynthesis in simple terms")
print(f"\nFinal response: {response}")

# Method 2: Using direct instantiation for more control
direct_agent = Agent(
    llm=client,
    max_steps=5,
    verbose=True
)

print("\n" + "="*50 + "\n")
print("Direct instantiation example:")
response = direct_agent.run("What is quantum computing?")
print(f"Response: {response}")

### When to Use Each Approach

**Use `create_agent()` when:**
- You want quick setup with defaults
- You need built-in validation
- You're prototyping rapidly
- You want consistent configuration

**Use `Agent()` directly when:**
- You need custom agent classes
- You want fine-grained control
- You're building complex systems
- You need to override internal methods

# Create a tool registry
calculator_registry = ToolRegistry()

# Define calculator tools
@tool(registry=calculator_registry)
def add(a: float, b: float) -> float:
    """Add two numbers together."""
    return a + b

@tool(registry=calculator_registry)
def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

@tool(registry=calculator_registry)
def divide(a: float, b: float) -> float:
    """Divide the first number by the second."""
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b

@tool(registry=calculator_registry)
def power(base: float, exponent: float) -> float:
    """Raise base to the power of exponent."""
    return base ** exponent

# Method 1: Create agent with tools using factory function
calculator_agent = create_agent(
    "calculator assistant",
    model="gpt-4",
    tools=calculator_registry,
    verbose=True
)

# Method 2: Using direct instantiation
# calculator_agent = Agent(
#     llm=client,
#     tools=calculator_registry,
#     verbose=True
# )

# Test the calculator agent
print("Calculator Agent Demo:")
print("="*50)

questions = [
    "What is 25 + 37?",
    "Calculate 15 * 8",
    "What is 2 raised to the power of 10?",
    "If I have 100 dollars and spend 37, how much do I have left?"
]

for question in questions:
    print(f"\nQuestion: {question}")
    answer = calculator_agent.run(question)
    print(f"Answer: {answer}")
    print("-"*30)

In [ ]:
# Create a tool registry
calculator_registry = ToolRegistry()

# Define calculator tools
@tool(registry=calculator_registry)
def add(a: float, b: float) -> float:
    """Add two numbers together."""
    return a + b

@tool(registry=calculator_registry)
def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

@tool(registry=calculator_registry)
def divide(a: float, b: float) -> float:
    """Divide the first number by the second."""
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b

@tool(registry=calculator_registry)
def power(base: float, exponent: float) -> float:
    """Raise base to the power of exponent."""
    return base ** exponent

# Create an agent with calculator tools
calculator_agent = Agent(
    llm=client,
    tools=calculator_registry,
    verbose=True
)

# Test the calculator agent
print("Calculator Agent Demo:")
print("="*50)

questions = [
    "What is 25 + 37?",
    "Calculate 15 * 8",
    "What is 2 raised to the power of 10?",
    "If I have 100 dollars and spend 37, how much do I have left?"
]

for question in questions:
    print(f"\nQuestion: {question}")
    answer = calculator_agent.run(question)
    print(f"Answer: {answer}")
    print("-"*30)

# Create a more complex tool set
research_registry = ToolRegistry()

@tool(registry=research_registry)
def search_web(query: str) -> str:
    """Search the web for information."""
    # Mock search results
    results = {
        "python": "Python is a high-level programming language known for its simplicity.",
        "machine learning": "Machine learning is a subset of AI that enables systems to learn from data.",
        "climate change": "Climate change refers to long-term shifts in global temperatures and weather patterns.",
    }
    
    for key, value in results.items():
        if key in query.lower():
            return value
    
    return f"Found information about {query}: General information available."

@tool(registry=research_registry)
def get_current_date() -> str:
    """Get today's date."""
    return datetime.now().strftime("%Y-%m-%d")

@tool(registry=research_registry)
def calculate_days_until(target_date: str) -> int:
    """Calculate days until a target date (YYYY-MM-DD format)."""
    target = datetime.strptime(target_date, "%Y-%m-%d")
    today = datetime.now()
    delta = target - today
    return delta.days

@tool(registry=research_registry)
def save_note(title: str, content: str) -> str:
    """Save a note for later reference."""
    # Mock saving
    return f"Note '{title}' saved successfully with {len(content)} characters."

# Create a research agent using factory function
research_agent = create_agent(
    "research assistant",
    model="gpt-4",
    tools=research_registry,
    max_steps=8,
    verbose=True
)

# Complex query that requires multiple steps
print("Research Agent Demo - Watch the ReAct pattern in action:")
print("="*50)

complex_query = """
I need to prepare for a Python conference. Can you:
1. Tell me what Python is
2. Find out today's date
3. Calculate how many days until 2024-06-15 (conference date)
4. Save a note with this information
"""

result = research_agent.run(complex_query)
print(f"\nFinal Result: {result}")

In [ ]:
# Create a more complex tool set
research_registry = ToolRegistry()

@tool(registry=research_registry)
def search_web(query: str) -> str:
    """Search the web for information."""
    # Mock search results
    results = {
        "python": "Python is a high-level programming language known for its simplicity.",
        "machine learning": "Machine learning is a subset of AI that enables systems to learn from data.",
        "climate change": "Climate change refers to long-term shifts in global temperatures and weather patterns.",
    }
    
    for key, value in results.items():
        if key in query.lower():
            return value
    
    return f"Found information about {query}: General information available."

@tool(registry=research_registry)
def get_current_date() -> str:
    """Get today's date."""
    return datetime.now().strftime("%Y-%m-%d")

@tool(registry=research_registry)
def calculate_days_until(target_date: str) -> int:
    """Calculate days until a target date (YYYY-MM-DD format)."""
    target = datetime.strptime(target_date, "%Y-%m-%d")
    today = datetime.now()
    delta = target - today
    return delta.days

@tool(registry=research_registry)
def save_note(title: str, content: str) -> str:
    """Save a note for later reference."""
    # Mock saving
    return f"Note '{title}' saved successfully with {len(content)} characters."

# Create a research agent
research_agent = Agent(
    llm=client,
    tools=research_registry,
    max_steps=8,
    verbose=True
)

# Complex query that requires multiple steps
print("Research Agent Demo - Watch the ReAct pattern in action:")
print("="*50)

complex_query = """
I need to prepare for a Python conference. Can you:
1. Tell me what Python is
2. Find out today's date
3. Calculate how many days until 2024-06-15 (conference date)
4. Save a note with this information
"""

result = research_agent.run(complex_query)
print(f"\nFinal Result: {result}")

# Create tools that work with context
context_registry = ToolRegistry()

# Simulated database
user_database = {
    "alice": {"age": 28, "city": "New York", "occupation": "Engineer"},
    "bob": {"age": 35, "city": "San Francisco", "occupation": "Designer"},
    "charlie": {"age": 42, "city": "London", "occupation": "Manager"}
}

@tool(registry=context_registry)
def lookup_user(username: str) -> dict:
    """Look up user information by username."""
    user = user_database.get(username.lower())
    if user:
        return {"found": True, "username": username, **user}
    return {"found": False, "username": username}

@tool(registry=context_registry)
def compare_ages(user1: str, user2: str) -> str:
    """Compare ages of two users."""
    info1 = user_database.get(user1.lower())
    info2 = user_database.get(user2.lower())
    
    if not info1 or not info2:
        return "One or both users not found"
    
    age_diff = abs(info1['age'] - info2['age'])
    older = user1 if info1['age'] > info2['age'] else user2
    
    return f"{older} is older. Age difference: {age_diff} years"

@tool(registry=context_registry)
def find_users_by_city(city: str) -> list:
    """Find all users in a specific city."""
    users = []
    for username, info in user_database.items():
        if info['city'].lower() == city.lower():
            users.append({"username": username, **info})
    return users

# Create context-aware agent using factory function
context_agent = create_agent(
    "user data analyst",
    model="gpt-4",
    tools=context_registry,
    verbose=True
)

# Query requiring context awareness
print("Context-Aware Agent Demo:")
print("="*50)

query = """
I need information about our users:
1. Who is older, Alice or Bob?
2. What cities do they live in?
3. Are there any other users in San Francisco?
"""

result = context_agent.run(query)
print(f"\nResult: {result}")

In [ ]:
# Create tools that work with context
context_registry = ToolRegistry()

# Simulated database
user_database = {
    "alice": {"age": 28, "city": "New York", "occupation": "Engineer"},
    "bob": {"age": 35, "city": "San Francisco", "occupation": "Designer"},
    "charlie": {"age": 42, "city": "London", "occupation": "Manager"}
}

@tool(registry=context_registry)
def lookup_user(username: str) -> dict:
    """Look up user information by username."""
    user = user_database.get(username.lower())
    if user:
        return {"found": True, "username": username, **user}
    return {"found": False, "username": username}

@tool(registry=context_registry)
def compare_ages(user1: str, user2: str) -> str:
    """Compare ages of two users."""
    info1 = user_database.get(user1.lower())
    info2 = user_database.get(user2.lower())
    
    if not info1 or not info2:
        return "One or both users not found"
    
    age_diff = abs(info1['age'] - info2['age'])
    older = user1 if info1['age'] > info2['age'] else user2
    
    return f"{older} is older. Age difference: {age_diff} years"

@tool(registry=context_registry)
def find_users_by_city(city: str) -> list:
    """Find all users in a specific city."""
    users = []
    for username, info in user_database.items():
        if info['city'].lower() == city.lower():
            users.append({"username": username, **info})
    return users

# Create context-aware agent
context_agent = Agent(
    llm=client,
    tools=context_registry,
    verbose=True
)

# Query requiring context awareness
print("Context-Aware Agent Demo:")
print("="*50)

query = """
I need information about our users:
1. Who is older, Alice or Bob?
2. What cities do they live in?
3. Are there any other users in San Francisco?
"""

result = context_agent.run(query)
print(f"\nResult: {result}")

## Custom Agent Behavior

Customize how agents think and act:

In [ ]:
# Create a specialized agent with custom system prompt
class SpecializedAgent(Agent):
    """Agent with specialized behavior."""
    
    def __init__(self, llm, tools=None, role="assistant", style="professional"):
        super().__init__(llm, tools)
        self.role = role
        self.style = style
    
    def _build_system_prompt(self) -> str:
        """Build custom system prompt."""
        base_prompt = super()._build_system_prompt()
        
        custom_prompt = f"""
{base_prompt}

You are a {self.role} who communicates in a {self.style} manner.
Always consider the user's needs and provide helpful, accurate responses.
When using tools, explain why you're using them.
"""
        return custom_prompt.strip()

# Create different specialized agents
data_tools = ToolRegistry()

@tool(registry=data_tools)
def analyze_numbers(numbers: list) -> dict:
    """Analyze a list of numbers."""
    if not numbers:
        return {"error": "No numbers provided"}
    
    return {
        "count": len(numbers),
        "sum": sum(numbers),
        "average": sum(numbers) / len(numbers),
        "min": min(numbers),
        "max": max(numbers)
    }

@tool(registry=data_tools)
def generate_report(data: dict, format: str = "simple") -> str:
    """Generate a report from data."""
    if format == "simple":
        return f"Report: {json.dumps(data, indent=2)}"
    elif format == "detailed":
        report = "Detailed Analysis Report\n" + "="*30 + "\n"
        for key, value in data.items():
            report += f"{key.capitalize()}: {value}\n"
        return report
    else:
        return "Unknown format"

# Create specialized agents
data_scientist = SpecializedAgent(
    llm=client,
    tools=data_tools,
    role="data scientist",
    style="analytical and precise"
)

teacher = SpecializedAgent(
    llm=client,
    tools=data_tools,
    role="mathematics teacher",
    style="patient and educational"
)

# Test different agent personalities
test_query = "Analyze these test scores: [85, 92, 78, 95, 88, 91, 83, 87, 90, 86]"

print("Data Scientist Agent:")
print("="*50)
scientist_response = data_scientist.run(test_query)
print(scientist_response)

print("\n\nTeacher Agent:")
print("="*50)
teacher_response = teacher.run(test_query)
print(teacher_response)

## Error Handling and Recovery

Agents can handle errors gracefully:

In [ ]:
# Create tools that might fail
unreliable_registry = ToolRegistry()

@tool(registry=unreliable_registry)
def risky_operation(value: int) -> str:
    """A tool that might fail."""
    if value < 0:
        raise ValueError("Negative values not supported")
    if value > 100:
        raise ValueError("Value too large")
    return f"Successfully processed value: {value}"

@tool(registry=unreliable_registry)
def safe_fallback(value: int) -> str:
    """A safer alternative tool."""
    return f"Safely handled value: {abs(value)}"

@tool(registry=unreliable_registry)
def validate_input(value: int) -> bool:
    """Check if input is valid."""
    return 0 <= value <= 100

# Create an agent that handles errors
resilient_agent = Agent(
    llm=client,
    tools=unreliable_registry,
    max_steps=10,
    verbose=True
)

# Test error handling
print("Resilient Agent Demo:")
print("="*50)

test_cases = [
    "Process the value 50",
    "Process the value -10",
    "Process the value 150"
]

for test in test_cases:
    print(f"\nTest: {test}")
    print("-"*30)
    result = resilient_agent.run(test)
    print(f"Result: {result}")
    print("="*50)

## Multi-Agent Systems

Create systems where multiple agents work together:

In [ ]:
class MultiAgentSystem:
    """Coordinate multiple specialized agents."""
    
    def __init__(self):
        self.agents = {}
        self.shared_context = {}
    
    def add_agent(self, name: str, agent: Agent):
        """Add an agent to the system."""
        self.agents[name] = agent
    
    def delegate(self, task: str, agent_name: str) -> str:
        """Delegate a task to a specific agent."""
        if agent_name not in self.agents:
            return f"Agent {agent_name} not found"
        
        agent = self.agents[agent_name]
        result = agent.run(task)
        
        # Store result in shared context
        self.shared_context[agent_name] = result
        
        return result
    
    def collaborate(self, task: str, agent_sequence: list) -> dict:
        """Have agents collaborate in sequence."""
        results = {}
        current_context = task
        
        for agent_name in agent_sequence:
            if agent_name not in self.agents:
                results[agent_name] = f"Agent not found"
                continue
            
            # Build context from previous results
            if results:
                context = f"{current_context}\n\nPrevious results: {json.dumps(results, indent=2)}"
            else:
                context = current_context
            
            # Get agent result
            result = self.agents[agent_name].run(context)
            results[agent_name] = result
        
        return results

# Create specialized agents for different tasks
research_tools = ToolRegistry()
writing_tools = ToolRegistry()
review_tools = ToolRegistry()

# Research tools
@tool(registry=research_tools)
def research_topic(topic: str) -> str:
    """Research a topic."""
    return f"Research findings on {topic}: Key points include historical context, current trends, and future outlook."

# Writing tools
@tool(registry=writing_tools)
def write_draft(topic: str, research: str = "") -> str:
    """Write a draft based on research."""
    return f"Draft about {topic}: Introduction, main body covering research points, and conclusion."

# Review tools
@tool(registry=review_tools)
def review_content(content: str) -> dict:
    """Review and score content."""
    return {
        "quality_score": 8.5,
        "suggestions": ["Add more examples", "Clarify technical terms"],
        "approval": "approved with minor revisions"
    }

# Create the multi-agent system
content_system = MultiAgentSystem()

# Add specialized agents
content_system.add_agent("researcher", Agent(llm=client, tools=research_tools))
content_system.add_agent("writer", Agent(llm=client, tools=writing_tools))
content_system.add_agent("reviewer", Agent(llm=client, tools=review_tools))

# Test collaboration
print("Multi-Agent Content Creation System:")
print("="*50)

task = "Create an article about the impact of AI on education"

results = content_system.collaborate(
    task,
    ["researcher", "writer", "reviewer"]
)

print("Collaboration Results:")
for agent, result in results.items():
    print(f"\n{agent.upper()}:")
    print(result)
    print("-"*30)

## Real-World Example: Customer Support Agent

Build a complete customer support agent system:

In [ ]:
# Customer support tools
support_registry = ToolRegistry()

# Mock customer database
customer_db = {
    "12345": {
        "name": "John Smith",
        "email": "john@example.com",
        "plan": "Premium",
        "issues": [
            {"date": "2024-01-10", "issue": "Login problem", "resolved": True},
            {"date": "2024-01-15", "issue": "Billing question", "resolved": True}
        ]
    },
    "67890": {
        "name": "Jane Doe",
        "email": "jane@example.com",
        "plan": "Basic",
        "issues": []
    }
}

# Knowledge base
knowledge_base = {
    "password_reset": "To reset password: 1) Click 'Forgot Password' 2) Enter email 3) Check email for reset link",
    "billing": "Billing cycles are monthly. Premium: $99/mo, Basic: $29/mo. Cancel anytime.",
    "features": "Premium includes: unlimited storage, priority support, advanced analytics",
    "refund": "30-day money-back guarantee. Contact support for refund processing."
}

@tool(registry=support_registry)
def lookup_customer(customer_id: str) -> dict:
    """Look up customer information by ID."""
    customer = customer_db.get(customer_id)
    if customer:
        return {"found": True, "customer": customer}
    return {"found": False, "message": "Customer not found"}

@tool(registry=support_registry)
def search_knowledge_base(query: str) -> str:
    """Search the knowledge base for information."""
    query_lower = query.lower()
    
    for topic, info in knowledge_base.items():
        if topic in query_lower or any(word in query_lower for word in topic.split('_')):
            return f"Found information: {info}"
    
    return "No specific information found. Please provide more details."

@tool(registry=support_registry)
def create_ticket(customer_id: str, issue: str, priority: str = "normal") -> dict:
    """Create a support ticket."""
    ticket = {
        "ticket_id": f"TKT-{datetime.now().strftime('%Y%m%d%H%M%S')}",
        "customer_id": customer_id,
        "issue": issue,
        "priority": priority,
        "status": "open",
        "created_at": datetime.now().isoformat()
    }
    return ticket

@tool(registry=support_registry)
def check_account_status(customer_id: str) -> dict:
    """Check customer account status."""
    customer = customer_db.get(customer_id)
    if not customer:
        return {"status": "not_found"}
    
    return {
        "status": "active",
        "plan": customer["plan"],
        "recent_issues": len([i for i in customer["issues"] if not i["resolved"]])
    }

@tool(registry=support_registry)
def send_email(to: str, subject: str, body: str) -> dict:
    """Send an email to customer."""
    return {
        "sent": True,
        "to": to,
        "subject": subject,
        "timestamp": datetime.now().isoformat()
    }

# Create customer support agent
class CustomerSupportAgent(Agent):
    def _build_system_prompt(self) -> str:
        return """
You are a helpful customer support agent. Your goals are to:
1. Understand the customer's issue
2. Look up relevant information
3. Provide helpful solutions
4. Create tickets for issues that need escalation
5. Always be polite and professional

When handling requests:
- First identify the customer if a customer ID is provided
- Search the knowledge base for common issues
- Create tickets for complex issues
- Send follow-up emails when appropriate
"""

support_agent = CustomerSupportAgent(
    llm=client,
    tools=support_registry,
    verbose=True,
    max_steps=10
)

# Test the support agent
print("Customer Support Agent Demo:")
print("="*50)

# Test case 1: Password reset
print("\nCase 1: Password Reset")
print("-"*30)
response = support_agent.run(
    "I'm customer 12345 and I forgot my password. Can you help?"
)
print(f"\nAgent Response: {response}")

# Test case 2: Billing inquiry
print("\n\nCase 2: Billing Question")
print("-"*30)
response = support_agent.run(
    "Customer ID 67890 here. I want to know about upgrading to Premium. What's included and how much does it cost?"
)
print(f"\nAgent Response: {response}")

# Test case 3: Complex issue
print("\n\nCase 3: Complex Technical Issue")
print("-"*30)
response = support_agent.run(
    "I'm customer 12345. The analytics dashboard is showing incorrect data for the last week. This is affecting my business reports!"
)
print(f"\nAgent Response: {response}")

## Debugging and Monitoring Agents

Tools for understanding agent behavior:

In [ ]:
class DebugAgent(Agent):
    """Agent with enhanced debugging capabilities."""
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.step_history = []
        self.tool_calls = []
        self.reasoning_steps = []
    
    def run(self, task: str) -> str:
        """Run with detailed logging."""
        self.step_history = []
        self.tool_calls = []
        self.reasoning_steps = []
        
        print("🚀 Starting Agent Execution")
        print(f"📋 Task: {task}")
        print("="*50)
        
        result = super().run(task)
        
        print("\n" + "="*50)
        print("📊 Execution Summary:")
        print(f"Total steps: {len(self.step_history)}")
        print(f"Tool calls: {len(self.tool_calls)}")
        print(f"Final result length: {len(result)} characters")
        
        return result
    
    def get_execution_report(self) -> dict:
        """Get detailed execution report."""
        return {
            "total_steps": len(self.step_history),
            "tool_calls": self.tool_calls,
            "reasoning_summary": self.reasoning_steps,
            "success": True if self.step_history else False
        }

# Create debug tools
debug_registry = ToolRegistry()

@tool(registry=debug_registry)
def process_data(data: str) -> str:
    """Process some data."""
    return f"Processed: {data.upper()}"

@tool(registry=debug_registry)
def validate_data(data: str) -> bool:
    """Validate data."""
    return len(data) > 0 and data.isalnum()

# Create and run debug agent
debug_agent = DebugAgent(
    llm=client,
    tools=debug_registry,
    verbose=False  # We'll do custom logging
)

# Run with debugging
result = debug_agent.run("Process the data 'hello123' if it's valid")
print(f"\n✅ Result: {result}")

# Get execution report
report = debug_agent.get_execution_report()
print("\n📈 Execution Report:")
print(json.dumps(report, indent=2))

# Best practices for agents

# 1. Design focused tools
good_tools = ToolRegistry()

@tool(registry=good_tools)
def get_user_by_id(user_id: int) -> dict:
    """Get user by ID - single responsibility."""
    # Focused on one task
    return {"id": user_id, "name": "User"}

# Not this:
# @tool
# def do_everything(action, data):
#     # Too many responsibilities!
#     pass

# 2. Provide clear tool descriptions
@tool(registry=good_tools)
def calculate_shipping_cost(weight_kg: float, distance_km: float) -> float:
    """Calculate shipping cost based on weight and distance.
    
    Args:
        weight_kg: Package weight in kilograms
        distance_km: Shipping distance in kilometers
        
    Returns:
        Shipping cost in USD
    """
    base_rate = 5.0
    weight_rate = 0.5 * weight_kg
    distance_rate = 0.1 * distance_km
    return base_rate + weight_rate + distance_rate

# 3. Handle edge cases in tools
@tool(registry=good_tools)
def safe_string_operation(text: str, operation: str = "upper") -> str:
    """Safely perform string operations."""
    if not text:
        return "Empty input"
    
    if operation == "upper":
        return text.upper()
    elif operation == "lower":
        return text.lower()
    elif operation == "title":
        return text.title()
    else:
        return f"Unknown operation: {operation}"

# 4. Set appropriate agent parameters using factory function
well_configured_agent = create_agent(
    "test assistant",
    model="gpt-4",
    tools=good_tools,
    max_steps=8,      # Reasonable limit
    verbose=True      # For development/debugging
)

# Alternative: Direct instantiation
# well_configured_agent = Agent(
#     llm=client,
#     tools=good_tools,
#     max_steps=8,
#     verbose=True
# )

# 5. Test agents thoroughly
def test_agent(agent, test_cases):
    """Test an agent with multiple cases."""
    results = []
    for test in test_cases:
        try:
            result = agent.run(test["input"])
            results.append({
                "test": test["name"],
                "success": True,
                "result": result
            })
        except Exception as e:
            results.append({
                "test": test["name"],
                "success": False,
                "error": str(e)
            })
    return results

# Example test cases
test_cases = [
    {"name": "Simple calculation", "input": "Calculate shipping for 5kg package going 100km"},
    {"name": "String operation", "input": "Convert 'hello world' to uppercase"},
    {"name": "User lookup", "input": "Get information for user ID 123"}
]

print("Testing agent with multiple cases:")
test_results = test_agent(well_configured_agent, test_cases)
for result in test_results:
    print(f"\n{result['test']}: {'✅' if result['success'] else '❌'}")
    if result['success']:
        print(f"Result: {result['result'][:100]}...")
    else:
        print(f"Error: {result['error']}")

In [ ]:
## Summary

In this tutorial, you learned:

- ✅ How to create agents using the simplified `create_agent()` factory function
- ✅ How to create agents with direct instantiation for more control
- ✅ How the ReAct pattern works
- ✅ How to provide agents with tools
- ✅ How to customize agent behavior
- ✅ How to handle errors and edge cases
- ✅ How to build multi-agent systems
- ✅ How to debug and monitor agents
- ✅ Best practices for agent design

Agents are powerful for:
- Automating complex workflows
- Building intelligent assistants
- Creating autonomous systems
- Solving multi-step problems

## Next Steps

Continue with:

- **Tutorial 9: Advanced Features** - Async operations, streaming, and more
- **Tutorial 10: Real-World Examples** - Complete applications
- **Tutorial 11: Simplified API** - Learn more about the new factory functions

Happy agent building! 🤖

## Summary

In this tutorial, you learned:

- ✅ How to create basic agents
- ✅ How the ReAct pattern works
- ✅ How to provide agents with tools
- ✅ How to customize agent behavior
- ✅ How to handle errors and edge cases
- ✅ How to build multi-agent systems
- ✅ How to debug and monitor agents
- ✅ Best practices for agent design

Agents are powerful for:
- Automating complex workflows
- Building intelligent assistants
- Creating autonomous systems
- Solving multi-step problems

## Next Steps

Continue with:

- **Tutorial 9: Advanced Features** - Async operations, streaming, and more
- **Tutorial 10: Real-World Examples** - Complete applications

Happy agent building! 🤖